# Статистические модели прогнозирования


Разные статистические модели прогнозирования тесно связаны друг с другом: в целях улучшения результата более простые подходы расширялись и становились основой для более сложных. В итоге в статистике появилось несколько методов прогнозирования, отличающихся между собой деталями, которые мы рассмотрим в этом юните.

СКОЛЬЗЯЩЕЕ СРЕДНЕЕ

В предыдущем модуле мы уже знакомились с экспоненциальным сглаживанием и получением с его помощью прогноза. Помимо простого экспоненциального сглаживания, есть также простое сглаживание, или скользящее среднее.

Для получения элементарного случая скользящего среднего проще всего взять среднее арифметическое двух последних наблюдений. Мы получим новый временной ряд, каждый член которого — среднее арифметическое двух соседних значений исходного ряда:



Чуть более продвинутый способ — усреднить сразу несколько наблюдений. Это так называемое простое скользящее среднее (Simple Moving Average, SMA):



Таким образом, в скользящем среднем мы суммируем несколько последовательных точек временного ряда и делим эту сумму на количество самих точек, то есть считаем математическое усреднение за определённый период.

Количество точек для суммирования определяется размером окна (q). Чем больше размер, тем больше данные сглаживаются.

Для сглаживания мы будем использовать встроенный метод pandas.Series.rolling() — он принимает на вход параметр window и ожидает после себя агрегирующую функцию для сглаживания (обычно используется среднее). Из преимуществ этого метода можно отметить простоту реализации и интерпретации, из недостатков — чувствительность.

Посмотрим на некоторый временной ряд и результаты применения сглаживания к нему:

Скользящее среднее с узким окном (размер окна — два дня) неэффективно борется с выбросами.


Скользящее среднее с широким окном (размер окна — 30 дней) может привести к потере информации, сгладив полезную информацию.


На данном графике с окном размера 15 есть прослеживающийся период и изменение амплитуды с течением времени.

Модель скользящего среднего порядка (ширина окна) q обозначается как MA(q), а предсказание строится как значение белого шума в момент времени t () + скользящее среднее с окном размера q (вычисляется как ):


где  и  — неизвестные параметры, которые настраиваются в процессе обучения.

Скользящее среднее редко используется как самостоятельная модель, но иногда помогает избавить ряд от выбросов и лишнего шума, чтобы упростить визуальный анализ.

## ARMA И ARIMA

Что получится, если объединить некоторые из изученных нами методов? Уже есть предположения, что такое ARMA?

Если вы внимательно следили за ходом событий, то смогли догадаться, что ARMA — это авторегрессионное скользящее среднее, или модель авторегрессии-скользящего среднего. В ней p авторегрессионных слагаемых и q слагаемых скользящего среднего шумовой компоненты:

img

Таким образом ARMA объединяет преимущества двух ранее изученных методов и имеет два параметра:

p — параметр авторегрессионной модели, который мы учились определять в предыдущем модуле (AR(p));
q — параметр скользящего среднего (MA(q)).
Параметр p мы определяли по графику частичной автокорреляции. Параметр q для скользящего среднего определяют так же, но по коррелограмме (графику автокорреляции).

«Было бы здорово применить всё это и для временных рядов, которые не являются стационарными», — подумали Дж. Бокс и Г. Дженкинс и расширили модель ARMA до ARIMA.

ARIMA расшифровывается как Autoregressive Integrated Moving Average и включает в себя ещё один параметр (d), который означает, что дифференцирование временного ряда порядка d приводит ряд к стационарности и будет подчиняться модели ARMA.

d — это тот самый порядок дифференцирования из предыдущего модуля, который приводил нестационарный ряд к стационарности. Это значит, что даже если наш ряд нестационарный, мы можем сделать его стационарным путём взятия разностей. Запомнив получившееся количество дифференцирований, можно смело применять к нему ARIMA.

И ARMA, и ARIMA реализованы на Python в классе ARIMA из statsmodels. Данному классу необходимо передать в качестве параметров временной ряд и порядок order (ARIMA(dta, order=(2, 0, 0))). Для параметра order нужно указать p, d и q (именно в таком порядке), причём для получения ARMA необходимо указать d=0.

Резюмируем:

Если ряд стационарный, используем ARMA.
Если ряд нестационарный (имеет тренд), с помощью дифференцирования определяем порядок d и используем ARIMA.
Теперь мы можем проверять временные ряды из задач на стационарность и вне зависимости от результата применять к ним одну из статистических моделей.

SARIMA

Модель ARIMA отлично учитывает и тренд (благодаря скользящему среднему), и зависимость от предыдущих значений (благодаря авторегрессии), но в ней не хватает учёта сезонности. В таком случае можно добавить к ARIMA учёт сезонности, и тогда мы получим следующую модель — сезонную ARIMA, или SARIMA (Seasonal ARIMA).

Эта модель очень похожа на ARIMA, за исключением того, что в ней есть дополнительный набор компонентов авторегрессии и скользящего среднего.

SARIMA позволяет различать данные по сезонной частоте, а также по их несезонным отличиям. Нахождение лучших для модели параметров можно упростить с помощью средств автоматического поиска, таких как auto_arima из pmdarima.

SARIMAX И ARIMAX

Вы уже могли заметить, что изученные нами статистические модели совершают предсказания, основываясь только на данных самого временного ряда. Но что если месяц или день недели тоже имеет значение? Или, например, на курс рубля, помимо даты и предыдущих значений, также влияют курсы других валют?

Последней статистической моделью, с которой мы познакомимся, будет SARIMAX. Её отличие от предыдущей версии заключается в том, что, помимо данных временного ряда, она учитывает экзогенные переменные. Таким образом мы сможем учитывать не только зависимости внутри данных, но и внешние факторы.

Для запуска моделей SARIMA и SARIMAX на Python нужно воспользоваться классом SARIMAX. Если вы хотите использовать SARIMA, необходимо задать два обязательных параметра — order и seasonal_order.

order — это порядок для модели (ARIMA(p, d, q)). В seasonal_order необходимо передать ещё четыре параметра:

P — сезонный авторегрессионный порядок;
D — порядок дифференцирования сезонного ряда;
Q — порядок сезонной скользящей средней;
m — размер сезонного периода.
Если размер сезонного периода m можно определить по сезонной компоненте (мы уже раскладывали ряд на компоненты ранее — seasonal_decompose), то остальные параметры удобнее определять автоперебором (мы применим этот способ на практике в следующем юните).

Для учёта экзогенных переменных необходимо передать в класс SARIMAX параметр exog=x. В x должны находиться другие временные ряды, например курс доллара (x), который может влиять на курс рубля (y), или пометка, является ли каждая из дат праздничным днём.

Хороший пример реализации SARIMAХ приведён в официальной документации.

Обратите внимание, что также существует модель ARIMAX. Уже есть предположения, когда использовать эту модель? В отличие от SARIMAX, ARIMAX не учитывает сезонную составляющую, но имеет все преимущества ARIMA и учитывает экзогенные переменные.

КАК СРАВНИВАТЬ ЭТИ МОДЕЛИ?

Одним из распространённых способов является сравнение качества моделей по критерию Акаике (AIC). Этот информационный критерий вознаграждает модель за качество приближения обученного временного ряда к фактическому, а также «штрафует» её за использование излишнего количества параметров. Принято считать, что модель с наименьшим значением критерия AIC является наилучшей.

Для оценки модели критерием AIC необязательно пользоваться дополнительными методами. Этот критерий, как и другая информация, отображается после обучения модели при вызове встроенного метода fit_model.summary().

## Интерполяция и сэмплирование

✍ Иногда можно столкнуться с отсутствием данных, например когда вы обрабатываете показания счётчика потребления горячей и холодной воды, а он по какой-то причине не работал несколько дней. Пропуски могут быть заполнены значениями NaN или нулями.

Взгляните на график ниже: за два дня (конец декабря и начало января) данные отсутствуют (они равны нулю) — из-за этого образуются спады до 0 в районе января.


Хотим ли мы, чтобы модель, которая должна предсказывать объём потребления воды, учитывала эти значения, равные нулю? Очевидно, нет. Значит, нам нужно каким-то образом заполнить пропуски, причём заполнение нулями нам не подходит, а заполнение средним — тоже не самая эффективная идея.

А что, если у нас есть показания счётчика по дням, а заказчик хочет получить прогноз по часам? Тогда нам снова нужно как-то заполнить возникающие пропуски для часов.
Знакомьтесь — перед нами проблемы upsampling и downsampling на временных рядах.

Upsampling — это увеличение частоты выборки (повышение частоты дискретизации), например с минут до секунд. Также upsampling применяют для заполнения пропусков неизвестных значений. Для этой цели мы будем использовать интерполяцию.

Downsampling — это уменьшение частоты выборки, например с дней до месяцев.

DOWNSAMPLING

Начнём с простого — с downsampling.

По своей сути, downsampling — это перегруппировка. Мы можем сгруппировать значения, полученные по дням, в значения, полученные за месяц, путём использования метода groupby(). Однако существует ещё один встроенный в DataFrame метод с чуть более широким функционалом — resample(). Этот метод позволяет делать нестандартные группировки, такие как «за три дня» или «за каждые шесть секунд», то есть вы можете выбрать свой интервал группировки и получить результат, написав одну строку кода. Вызывать метод resample необходимо у датафрейма, в качестве индекса у которого используются даты в формате datetime. Например, если наш временной ряд с показаниями счетчика по потреблению воды изначально сгруппирован по дням, применив к нему resample, можно получить:

Дальше уже можно работать над построением модели.

Правила группировки (Q, H, M и др.) — это сокращения, используемые в работе с типом данных Timestamp. Полный перечень правил группировки можно найти в документации (таблица Date Offset).

Теперь пора переходить к upsampling. Но прежде рассмотрим понятие, которое мы уже упоминали выше, — «интерполяция».

ИНТЕРПОЛЯЦИЯ

Говоря простым языком, интерполяция — это нахождение некоторых промежуточных значений по функции, описывающей поведение данных. То есть если мы найдём такую функцию, значения которой будут совпадать с уже известными нам значениями, то можно предположить, что она поможет верно или приблизительно восстановить для нас неизвестные значения.

Существуют несколько способов нахождения этой функции, и интерполяция может быть выполнена с помощью:

линейной функции,
многочлена Лагранжа,
интерполяционной формулы Ньютона,
семейства сплайн-функций.

UPSAMPLING

Для реализации upsampling на практике мы будем использовать уже известный нам метод resample, чтобы декомпозировать данные, например от дня к часам, а затем воспользуемся встроенным методом interpolate(), который принимает в качестве аргумента указание метода интерполяции: 'linear', 'nearest', 'spline', 'barycentric', 'polynomial' и другие. То есть сначала мы применяем к данным resample() с параметром h (час), а затем вместо агрегирующей функции sum/mean и др. применяем interpolate('linear'), чтобы выполнить линейную интерполяцию.

Пример результата выполнения функции — ниже. Мы можем применить её к одному из ранее использованных в модуле временных рядов.

PROPHET

Prophet — это метод прогнозирования данных временных рядов на основе AR-модели, в которой учтены годовая, еженедельная и ежедневная сезонности, а также эффекты праздничных дней.

Prophet лучше всего работает с временными рядами, которые имеют сильные сезонные эффекты, а данные накоплены за несколько сезонов. Алгоритм устойчив к отсутствующим данным и сдвигам в тренде и обычно хорошо справляется с выбросами.

Prophet — библиотека с открытым исходным кодом, выпущенная командой Facebook Core Data Science. Для загрузки метод также доступен в PyPI (через pip install).

Prophet следует API модели sklearn. Это значит, что мы можем пользоваться им и его методами так же, как и в случае с моделями из sklearn: для инициализации модели создаётся экземпляр класса Prophet (myModel = Prophet()), а затем вызываются его методы обучения (.fit()) и прогнозирования (.predict()). Входные данные для методов Prophet должны представлять собой датафрейм с двумя столбцами — DS и Y.

Столбец DS (отметка даты) должен иметь временной формат (DateTime), например ГГГГ-ММ-ДД — для даты или ГГГГ-ММ-ДД ЧЧ:ММ:СС — для отметки времени.
Столбец Y должен быть числовым и представлять измерение, которое мы хотим спрогнозировать.

В качестве примера давайте рассмотрим временной ряд ежедневных просмотров страницы футболиста Пейтона Мэннинга на Wikipedia.

Импортируем Prophet и другие необходимые библиотеки, а также считаем датасет.



In [6]:
!pip show Prophet

Name: prophet
Version: 1.1.4
Summary: Automatic Forecasting Procedure
Home-page: 
Author: 
Author-email: "Sean J. Taylor" <sjtz@pm.me>, Ben Letham <bletham@fb.com>
License: MIT
Location: /Users/egor/anaconda3/envs/myenv/lib/python3.9/site-packages
Requires: cmdstanpy, convertdate, holidays, importlib-resources, LunarCalendar, matplotlib, numpy, pandas, python-dateutil, tqdm
Required-by: 


In [8]:
import pandas as pd
from prophet import Prophet

In [9]:
url = 'https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv'

df = pd.read_csv(url)

df.head()

,ds,y
0,2007-12-10,9.590761
1,2007-12-11,8.519590
2,2007-12-12,8.183677
3,2007-12-13,8.072467
4,2007-12-14,7.893572


Обучим модель на датасете:

In [10]:
model = Prophet()
model.fit(df)

15:43:48 - cmdstanpy - INFO - Chain [1] start processing
15:43:48 - cmdstanpy - INFO - Chain [1] done processing
15:43:48 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6
Optimization terminated abnormally. Falling back to Newton.
15:43:48 - cmdstanpy - INFO - Chain [1] start processing
15:43:48 - cmdstanpy - INFO - Chain [1] done processing
15:43:48 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6


RuntimeError: Error during optimization! Command '/Users/egor/anaconda3/envs/myenv/lib/python3.9/site-packages/prophet/stan_model/prophet_model.bin random seed=37047 data file=/var/folders/fg/7cyb3m_113j_1hkfwhtbsy8h0000gn/T/tmphlsyn3xf/w582g72_.json init=/var/folders/fg/7cyb3m_113j_1hkfwhtbsy8h0000gn/T/tmphlsyn3xf/i81gkf0p.json output file=/var/folders/fg/7cyb3m_113j_1hkfwhtbsy8h0000gn/T/tmphlsyn3xf/prophet_modeljbzmxa1q/prophet_model-20230721154348.csv method=optimize algorithm=newton iter=10000' failed: console log output:

dyld[55394]: Library not loaded: @rpath/libtbb.dylib
  Referenced from: <C1D45D90-9855-331A-ADB0-3D8F989802A1> /Users/egor/anaconda3/envs/myenv/lib/python3.9/site-packages/prophet/stan_model/prophet_model.bin
  Reason: tried: '/private/var/folders/fg/7cyb3m_113j_1hkfwhtbsy8h0000gn/T/pip-install-84ipzupm/prophet_919035b7c0884bfea0bf0d25e48d233a/build/lib.macosx-11.1-arm64-cpython-39/prophet/stan_model/cmdstan-2.31.0/stan/lib/stan_math/lib/tbb/libtbb.dylib' (no suc

Для получения прогноза необходимо использовать DataFrame со столбцом DS, содержащим даты, для которых должен быть сделан прогноз. Вы можете получить такой DataFrame, используя вспомогательный метод Prophet.make_future_dataframe(), в который необходимо передать число дней для совершения прогноза. По умолчанию он также будет включать более ранние даты, поэтому мы также увидим, как обучалась модель.

In [ ]:
future_df = model.make_future_dataframe(periods=365)

future_df.tail()

: 

В результате построения прогноза каждому DS будет присвоено прогнозируемое значение (YHAT). Также в прогнозном датафрейме будут два новых столбца ('yhat_lower', 'yhat_upper'), в которых будут находиться верхняя и нижняя границы доверительного интервала:

In [ ]:
forecast_df = model.predict(future_df)
forecast_df[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

: 

: 